I still want to implement acrpo. 

https://research.vu.nl/ws/portalfiles/portal/266879340/A_Lambda_Free_Higher_Order_Recursive_Path_Order.pdf implement lfrpo

Ground nominal term orderings.

Equivariance means that X = Y can't be oriented. or add(X,Y) = add(Y,X)
But we're ground so we're probably better behaved

Rudi noted that slots are lot of like unification vars that never get narrowed and have a disjointness constraint.
https://egraphs.zulipchat.com/#narrow/channel/328972-general/topic/EGRAPHS.20Meeting.202025-03-20.3A.20Slotted.20E-Graphs/near/543056830

If `t =g= s` was useful, what about `t <g< s`. Hmm. Ordered modulo g. Then we could talk about x + y < y + x ?

`2 <(-7)< -4` using shift group. Yes...  also `2 <-8< -4` but not `2 <-6< -4`
Combininig the inequality union find with the group union find.

An ordered group? https://en.wikipedia.org/wiki/Linearly_ordered_group
https://ncatlab.org/nlab/show/ordered+group
https://en.wikipedia.org/wiki/Partially_ordered_group

`def perm_lpo(t, s) -> tuple[Order, G]:`


https://memoryleak47.github.io/slotted-egraphs-demystified-i/
https://medium.com/@mineralsteins/ultimate-code-compression-with-slotted-egraph-74f1859c9df5

So we need an unfailing variant of knuth bendix

renamings aren't bijective. They are 1-1.

https://www.cs.ru.nl/~cynthiakop/2024_isr/day2handout.pdf  Termination:
the higher-order recursive path ordering - Kop


Have I done dumb ground kb in python?

https://drops.dagstuhl.de/storage/00lipics/lipics-vol021-rta2013/LIPIcs.RTA.2013.319/LIPIcs.RTA.2013.319.pdf normalized completion revisited. 2013. Has a table of moidifed rules. hmm. acrpo better than ackbo?


https://www.jonmsterling.com/01JQ/ disentangling unfication and implciti coercions 
"nested pattern unification" https://andraskovacs.github.io/pdfs/wits23prez.pdf



In [ ]:
from dataclasses import dataclass
@dataclass(frozen=True)
class App():
    f : str
    args : tuple["Term", ...] = ()

    def __len__(self) -> int:
        return 1 + sum(len(arg) for arg in self.args)

    def __repr__(self) -> str:
        if self.args:
            return f"{self.f}({', '.join(repr(arg) for arg in self.args)})"
        else:
            return self.f

    def __le__(self, other: "App") -> bool:
        return self == other or self < other
    def __lt__(self, other: "App") -> bool:
        if self == other:
            return False
        ls,lo = len(self), len(other)
        if ls != lo:
            return ls < lo
        if self.f != other.f:
            return self.f < other.f
        for sa, oa in zip(self.args, other.args):
            if sa != oa:
                return sa < oa
        return False
    def subst(self, a, b):
        if self == a:
            return b
        return App(self.f, tuple(arg.subst(a,b) for arg in self.args))    

x = App("x")
y = App("y")
f = lambda a,b: App("f", (a,b))
assert f(x,y) > x
assert y > x
assert not x > y
assert not x < x

@dataclass
class KB():
    E : list[tuple[App,App]]
    R : list[tuple[App,App]]
    # Ordering

    def orient(self, n: int):
        (a,b) = self.E.pop(n)
        assert b != a
        if b > a:
            a,b = b,a
        self.R.append((a,b))
        return self
    
    def triv(self, n : int):
        (a,b) = self.E.pop(n)
        assert a == b
        return self
    
    def simp(self, en : int, rn : int):
        (ae,be) = self.E.pop(en)
        (ar,br) = self.R[rn]
        self.E.append((ae.subst(ar,br), be.subst(ar,br)))
        return self
    
    def deduce(self, u : App, rleft : int, rright: int):
        (arl,brl) = self.R[rleft]
        (arr,brr) = self.R[rright]
        self.E.append((u.subst(arl,brl),
                       u.subst(arr,brr)))
        return self
    
    def rsimp(self, rn : int, rm : int):
        (ar,br) = self.R[rn]
        (am,bm) = self.R.pop(rm)
        self.R.append((am, bm.subst(ar,br)))
        return self

    def lsimp(self, rn : int):
        # TODO: need to dignify with encompassment condition
        self.E.append(self.R.pop(rn))
        return self



kb = KB([(f(x,y), y), (f(x,y), x)], [])

kb
kb.orient(0)
kb.simp(0,0)
kb
kb.orient(0)
kb.rsimp(1,0)
kb.deduce(f(x,y), 0, 1)
kb.orient(0)
kb.lsimp(1)
kb.simp(0,0)
kb.simp(0,1)
kb.triv(0)

KB(E=[], R=[(y, x), (f(x, x), x)])

In [ ]:
# ok but it hsould be schedules

In [ ]:
from dataclasses import dataclass, field

counter = 0
def fresh_counter() -> int:
    global counter
    counter += 1
    return counter

@dataclass(frozen=True)
class Slot():
    name : int = field(default_factory=fresh_counter)

@dataclass(frozen=True)
class App():
    f : str
    args : tuple["Term", ...] = ()

type Term = App | Slot
